In [1]:
from huggingface_hub import login
hf_token = ''
login(hf_token)

C:\Users\jc\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\jc\.cache\huggingface\token
Login successful


In [2]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [3]:
# DEVICE_MAP='cpu'
DEVICE_MAP='cuda'
BASE_MODEL = 'meta-llama/Meta-Llama-3-8B'
FINE_TUNED_MODEL_DIR = './fine-tuning/fine_tuned_llama'
PEFT_MODEL_DIR = './fine-tuned-llama-3'
OUTPUT_DIR='fine-tuned-llama-3'

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map=DEVICE_MAP,
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained(FINE_TUNED_MODEL_DIR)

Loading checkpoint shards: 100%|██████████| 4/4 [00:34<00:00,  8.61s/it]


In [6]:
model = PeftModel.from_pretrained(model, FINE_TUNED_MODEL_DIR)
model = model.merge_and_unload()
model._hf_peft_config_loaded = False

C:\Users\jc\.conda\envs\tfm_fine_tuning\Lib\site-packages\peft\tuners\lora\bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [9]:
prompt = """
You are a powerful text-to-Python model.
Your job is to answer questions about a Python.
You are given a question regarding Python code.
You must output the Python code that answers the question tabulated correctly.
Only one response was allowed.
### Input:
Create a function to return fibonacci sequence.

### Response:
"""
output = pipe(
    prompt,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)
print(output[0]["generated_text"])


You are a powerful text-to-Python model.
Your job is to answer questions about a Python.
You are given a question regarding Python code.
You must output the Python code that answers the question tabulated correctly.
Only one response was allowed.
### Input:
Create a function to return fibonacci sequence.

### Response:
```
def fibo(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibo(n-1) + fibo(n-2)
```


In [11]:
model.save_pretrained(OUTPUT_DIR, safe_serialization=False)
tokenizer.save_pretrained(OUTPUT_DIR, safe_serialization=False)

('fine-tuned-llama-3-cpu\\tokenizer_config.json',
 'fine-tuned-llama-3-cpu\\special_tokens_map.json',
 'fine-tuned-llama-3-cpu\\tokenizer.json')

In [ ]:
model.push_to_hub(f'jucargoe/{OUTPUT_DIR}', safe_serialization=False, use_temp_dir=False)
tokenizer.push_to_hub(f'jucargoe/{OUTPUT_DIR}', safe_serialization=False, use_temp_dir=False)

C:\Users\jc\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jc\.cache\huggingface\hub\models--jucargoe--fine-tuned-llama-3-cpu. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-0